<a href="https://colab.research.google.com/github/HARISPOUNRAJ/Akaike/blob/main/Hashira_java.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install Java JDK
!apt-get install openjdk-17-jdk -y

# Verify Java installation
!java -version
!javac -version


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  fonts-dejavu-core fonts-dejavu-extra libatk-wrapper-java
  libatk-wrapper-java-jni libxt-dev libxtst6 libxxf86dga1
  openjdk-17-jdk-headless openjdk-17-jre openjdk-17-jre-headless x11-utils
Suggested packages:
  libxt-doc openjdk-17-demo openjdk-17-source visualvm libnss-mdns
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  | fonts-wqy-zenhei fonts-indic mesa-utils
The following NEW packages will be installed:
  fonts-dejavu-core fonts-dejavu-extra libatk-wrapper-java
  libatk-wrapper-java-jni libxt-dev libxtst6 libxxf86dga1 openjdk-17-jdk
  openjdk-17-jdk-headless openjdk-17-jre openjdk-17-jre-headless x11-utils
0 upgraded, 12 newly installed, 0 to remove and 35 not upgraded.
Need to get 125 MB of archives.
After this operation, 287 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/m

In [40]:
%%writefile ShamirSecretRecovery.java
import java.io.*;
import java.math.BigInteger;
import java.util.*;
import org.json.*;

public class ShamirSecretRecovery {

    static class Share {
        int index;
        BigInteger value;
        Share(int index, BigInteger value) {
            this.index = index;
            this.value = value;
        }
    }

    public static void main(String[] args) {
        try {
            BufferedReader br = new BufferedReader(new InputStreamReader(System.in));
            StringBuilder sb = new StringBuilder();
            String line;
            while ((line = br.readLine()) != null) sb.append(line);
            JSONObject json = new JSONObject(sb.toString());

            int n = json.getJSONObject("keys").getInt("n");
            int k = json.getJSONObject("keys").getInt("k");

            List<Share> shares = new ArrayList<>();
            for (String key : json.keySet()) {
                if (key.equals("keys")) continue;
                JSONObject obj = json.getJSONObject(key);
                int base = Integer.parseInt(obj.getString("base"));
                BigInteger value = new BigInteger(obj.getString("value"), base);
                shares.add(new Share(Integer.parseInt(key), value));
            }

            Share wrongShare = null;
            BigInteger secret = null;

            outer:
            for (int i = 0; i < shares.size(); i++) {
                for (int j = 0; j < shares.size(); j++) {
                    if (i == j) continue;

                    List<Share> candidateShares = new ArrayList<>();
                    for (int idx = 0; idx < shares.size(); idx++) {
                        if (idx != j) candidateShares.add(shares.get(idx));
                    }

                    List<Share> subset = candidateShares.subList(0, k);

                    try {
                        BigInteger computedSecret = lagrangeInterpolationAtZero(subset);
                        boolean consistent = true;
                        for (Share s : shares) {
                            if (!subset.contains(s)) {
                                BigInteger predicted = lagrangeInterpolationAtX(subset, s.index);
                                if (!predicted.equals(s.value)) {
                                    consistent = false;
                                    break;
                                }
                            }
                        }
                        if (consistent) {
                            secret = computedSecret;
                            wrongShare = shares.get(j);
                            break outer;
                        }
                    } catch (ArithmeticException e) {}
                }
            }

            if (secret != null && wrongShare != null) {
                System.out.println("Secret: " + secret);
                System.out.println("Wrong Share: " + wrongShare.index + " (" + wrongShare.value + ")");
            } else {
                System.out.println("Failed to recover secret or identify wrong share.");
            }

        } catch (Exception e) { e.printStackTrace(); }
    }

    private static BigInteger lagrangeInterpolationAtZero(List<Share> shares) {
        BigInteger result = BigInteger.ZERO;
        for (int i = 0; i < shares.size(); i++) {
            BigInteger xi = BigInteger.valueOf(shares.get(i).index);
            BigInteger yi = shares.get(i).value;
            BigInteger numerator = BigInteger.ONE;
            BigInteger denominator = BigInteger.ONE;

            for (int j = 0; j < shares.size(); j++) {
                if (i == j) continue;
                BigInteger xj = BigInteger.valueOf(shares.get(j).index);
                numerator = numerator.multiply(xj.negate());
                denominator = denominator.multiply(xi.subtract(xj));
            }

            result = result.add(yi.multiply(numerator).divide(denominator));
        }
        return result;
    }

    private static BigInteger lagrangeInterpolationAtX(List<Share> shares, int x) {
        BigInteger result = BigInteger.ZERO;
        BigInteger X = BigInteger.valueOf(x);

        for (int i = 0; i < shares.size(); i++) {
            BigInteger xi = BigInteger.valueOf(shares.get(i).index);
            BigInteger yi = shares.get(i).value;
            BigInteger numerator = BigInteger.ONE;
            BigInteger denominator = BigInteger.ONE;

            for (int j = 0; j < shares.size(); j++) {
                if (i == j) continue;
                BigInteger xj = BigInteger.valueOf(shares.get(j).index);
                numerator = numerator.multiply(X.subtract(xj));
                denominator = denominator.multiply(xi.subtract(xj));
            }
            result = result.add(yi.multiply(numerator).divide(denominator));
        }
        return result;
    }
}


Overwriting ShamirSecretRecovery.java


In [33]:
!wget -O json.jar https://repo1.maven.org/maven2/org/json/json/20210307/json-20210307.jar


--2025-08-30 06:23:12--  https://repo1.maven.org/maven2/org/json/json/20210307/json-20210307.jar
Resolving repo1.maven.org (repo1.maven.org)... 199.232.192.209, 199.232.196.209, 2a04:4e42:4c::209, ...
Connecting to repo1.maven.org (repo1.maven.org)|199.232.192.209|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 69710 (68K) [application/java-archive]
Saving to: ‘json.jar’

json.jar            100%[===================>]  68.08K  --.-KB/s    in 0.004s  

2025-08-30 06:23:12 (15.8 MB/s) - ‘json.jar’ saved [69710/69710]



In [34]:
!javac -cp json.jar ShamirSecretRecovery.java


In [41]:
%%writefile input.json
{
  "keys": {"n":4,"k":3},
  "1": {"base":"10","value":"4"},
  "2": {"base":"2","value":"111"},
  "3": {"base":"10","value":"12"},
  "6": {"base":"4","value":"213"}
}


Overwriting input.json


In [36]:
!java -cp .:json.jar ShamirSecretRecovery < input.json


Secret: 2
Wrong Share: 2 (7)


In [37]:
{
  "keys": {
    "n": 10,
    "k": 7
  },
  "1": {
    "base": "6",
    "value": "13444211440455345511"
  },
  "2": {
    "base": "15",
    "value": "aed7015a346d635"
  },
  "3": {
    "base": "15",
    "value": "6aeeb69631c227c"
  },
  "4": {
    "base": "16",
    "value": "e1b5e05623d881f"
  },
  "5": {
    "base": "8",
    "value": "316034514573652620673"
  },
  "6": {
    "base": "3",
    "value": "2122212201122002221120200210011020220200"
  },
  "7": {
    "base": "3",
    "value": "20120221122211000100210021102001201112121"
  },
  "8": {
    "base": "6",
    "value": "20220554335330240002224253"
  },
  "9": {
    "base": "12",
    "value": "45153788322a1255483"
  },
  "10": {
    "base": "7",
    "value": "1101613130313526312514143"
  }
}


{'keys': {'n': 10, 'k': 7},
 '1': {'base': '6', 'value': '13444211440455345511'},
 '2': {'base': '15', 'value': 'aed7015a346d635'},
 '3': {'base': '15', 'value': '6aeeb69631c227c'},
 '4': {'base': '16', 'value': 'e1b5e05623d881f'},
 '5': {'base': '8', 'value': '316034514573652620673'},
 '6': {'base': '3', 'value': '2122212201122002221120200210011020220200'},
 '7': {'base': '3', 'value': '20120221122211000100210021102001201112121'},
 '8': {'base': '6', 'value': '20220554335330240002224253'},
 '9': {'base': '12', 'value': '45153788322a1255483'},
 '10': {'base': '7', 'value': '1101613130313526312514143'}}

In [39]:
!java -cp .:json.jar ShamirSecretRecovery < input_test2.json


Failed to recover secret or identify wrong share.
